In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pandasql as ps
import glob
import os

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [ ]:
path = '/datasets/real-estate-stillwater_20241001/'
all_files = glob.glob(os.path.join(path, "*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [ ]:
!ls /datasets/real-estate-stillwater_20241001/

In [ ]:
df.shape

In [ ]:
q1 = """SELECT sold_price FROM df ORDER BY 1 DESC LIMIT 100"""
print(ps.sqldf(q1, locals()))


In [ ]:
# Filter out everything but sold houses
df_sold = df.loc[df['status'] == 'SOLD']


# Drop rows will nulls
df_sold = df_sold.dropna(axis=0, subset=['lot_sqft','sold_price'])

# Drop outliers
df_sold.drop(df_sold[df_sold.lot_sqft > 1.5e7].index, inplace=True)
df_sold.drop(df_sold[df_sold.sold_price > 1.5e7].index, inplace=True)

# Normalize Price
print(f"Max home price: {max(df_sold['sold_price'])}")
df_sold['sold_price'] = df_sold['sold_price']/max(df_sold['sold_price'])

# Normalize lot size
print(f"Max lot size: {max(df_sold['lot_sqft'])}")
df_sold['lot_sqft'] = df_sold['lot_sqft']/max(df_sold['lot_sqft'])

In [ ]:
df.columns

In [ ]:
col_train = ['lot_sqft']
col_label = ['sold_price']
train_cols = col_train + col_label

df_train = df_sold[train_cols]
# df_train.head()

In [ ]:
df_exp = df_sold[train_cols]
df_exp = df_exp.dropna(axis=0, subset=train_cols)

In [ ]:
msk = np.random.rand(len(df_exp)) < 0.8

(training_data, training_labels) = df_exp[col_train][msk], df_exp[col_label][msk]
(test_data, test_labels) = df_exp[col_train][~msk], df_exp[col_label][~msk]

In [ ]:
l0 = Dense(units=1, input_shape=[1])
# model = Sequential([l0])
model = Sequential([
    Flatten(input_shape=(1, 1)),
    Dense(1, activation=tf.nn.relu),
    Dense(1, activation=tf.nn.softmax)
])
model.compile(optimizer='sgd', loss='mean_squared_error')

In [ ]:
# class acc_callback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if(logs.get('accuracy') > 0.98):
#            print("\nReached 98% accuracy, stopping training")
#            self.model.stop_training = True
# callback = acc_callback()

# model.fit(training_data, training_labels, epochs=5, callbacks=[callback])
model.fit(training_data, training_labels, epochs=5)

In [ ]:
model.evaluate(test_data, test_labels)

In [ ]:
classifications = model.predict(test_data)
print(classifications)
print(test_labels)


In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Flatten(input_shape=(1, 1)),
#     tf.keras.layers.Dense(1, activation=tf.nn.relu),
#     tf.keras.layers.Dense(1, activation=tf.nn.softmax)
# ])

In [ ]:
# class acc_callback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if(logs.get('accuracy') > 0.92):
#            print("\nReached 92% accuracy, stopping training")
#            self.model.stop_training = True
# callback = acc_callback()

In [ ]:
# model.fit(training_data, training_labels, epochs=5, callbacks=[callback])

In [ ]:
# model.compile(optimizer='adam',
#                 loss='sparse_categorical_crossentropy',
#                 metrics=['accuracy'])

In [ ]:

plt.scatter(df['last_sold_date'], df['sold_price']/1000)
plt.show()